In [1]:
from os import listdir
from os.path import isfile
from fnmatch import fnmatch
from math import isnan, ceil
from pprint import pprint
from statistics import median
from re import match, search

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.font_manager import FontProperties

In [2]:
def dump_csv(df, filename):
    df.to_csv(filename, float_format='%.4f')
    return

In [18]:
ratios = ['1:0']
runs = [x for x in range(1, 3 + 1)] 
instances = [x for x in range(1, 2 + 1)]
threads = 1 
clients = [x for x in range(1, 3 + 1)]
vclients = [1, 4, 8, 16, 24, 32]
worker_threads = [8, 16, 32, 64]
mws = [x for x in range(1, 2 + 1)]

vclients = [4]
mw_multi_index = [ratios, worker_threads, vclients]
mw_len_index = len(ratios) * len(worker_threads) * len(vclients)

mw_names=['ratio', 'WTS', 'VCs']

mw_index = pd.MultiIndex.from_product(mw_multi_index, names=mw_names)
columns = ['Throughput (ops/s)', 'ResponseTime (ms)']

columns += ['Jobs', 'ServiceTime (ms)', 'WaitingTime (ms)', 'QueueSize']

df = pd.read_csv('../throughput_for_writes/out/middleware_data.csv', index_col=[0,1,2,3,4])
df1 = pd.DataFrame([], index=mw_index, columns=columns)

for ratio in ratios:
    for wt in worker_threads:
        for vc in vclients:
            row = []
            for c in columns:
                if 'ops/s' in c or 'Jobs' in c or 'Queue' in c:
                    row.append(df.unstack().loc[(ratio, wt, vc), c].T.sum().mean())
                else:
                    row.append(df.unstack().loc[(ratio, wt, vc), c].T.mean().mean())
            df1.loc[(ratio, wt, vc)] = row
            
#Service rate is considered to be the max TP for each WT config, since, it constitutes a reasonable lower bound. 

df1['ServiceRate (ops/s)'] = pd.Series()
df1['Rho'] = pd.Series()
df1['En_q'] = pd.Series()
df1['En'] = pd.Series()
df1['Ew'] = pd.Series()

service = False

for ratio in ratios:
    for wt in worker_threads:
        for vc in vclients:
            if service:
                df1.loc[(ratio, wt, vc), 'ServiceRate (ops/s)'] = df.unstack().loc[(ratio, wt, vc), 'Throughput (ops/s)'].T.sum().max()
            else:
                df1.loc[(ratio, wt, vc), 'ServiceRate (ops/s)'] = 1000 / (df.unstack().loc[(ratio, wt, vc), 'ServiceTime (ms)'].T.mean().mean() / (2 * wt))
            
print(df1)        
# Utilization
df1['Rho'] = df1['Throughput (ops/s)'] / df1['ServiceRate (ops/s)']
# Expected number of Jobs in the system (w + s)
df1['En'] = df1['Rho'] / (1 - df1['Rho'])
# Expected number of Jobs in the queue(w)
df1['En_q'] = pow(df1['Rho'], 2) / (1 - df1['Rho'])
# Expected AVG Waiting Time in the queue(w)
df1['Ew'] = df1['Rho'] * (1 / df1['ServiceRate (ops/s)']) / (1.0 - df1['Rho']) * 1000
# Expected AVG Response Time
df1['Er'] = (1.0 / (df1['ServiceRate (ops/s)'] * (1.0 - df1['Rho']))) * 1000

dump_csv(df1, 'out/mm1.csv')
print(df1)

              Throughput (ops/s) ResponseTime (ms)    Jobs ServiceTime (ms)  \
ratio WTS VCs                                                                 
1:0   8   4              5048.05           2.29167  350644             1.81   
      16  4              5402.07           2.10833  374491             1.87   
      32  4              5354.77           2.14167  371717            1.905   
      64  4              5332.74           2.17667  369963            1.935   

              WaitingTime (ms) QueueSize  ServiceRate (ops/s)  Rho  En_q  En  \
ratio WTS VCs                                                                  
1:0   8   4           0.388333         2          8839.779006  NaN   NaN NaN   
      16  4           0.141667         0         17112.299465  NaN   NaN NaN   
      32  4               0.14         0         33595.800525  NaN   NaN NaN   
      64  4           0.143333         0         66149.870801  NaN   NaN NaN   

               Ew  
ratio WTS VCs      
1:0 